In [ ]:
# dependencies
import re
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt

from zipfile import ZipFile
from matplotlib.pyplot import figure
from collections import Counter, defaultdict
from itertools import combinations

In [ ]:
# path to data
path = "./archive.zip"

In [ ]:
# extracting the compressed data
with ZipFile(path, 'r') as zip:
  zip.extractall()

In [ ]:
# loading the data
userData = pd.read_csv("./anonymized_disqus_user_data.csv")
botComments = pd.read_csv("./botnet_comments.csv")
potentialBot = pd.read_csv("./potential_social_botnets.csv")
metaData = pd.read_csv("./user_meta_data.csv")

In [ ]:
# converting message to string as some of them are float, etc.
botComments.message = botComments.message.astype(str)

In [ ]:
# removing the html tags
botComments.message = botComments.message.apply(lambda html: re.sub('<.*?>', '', html))

In [ ]:
# installing sentence transformers (https://github.com/UKPLab/sentence-transformers)
!pip install -U sentence-transformers

In [ ]:
# dependencies
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
# loading the corpusEmbeddings
corpusEmbeddings = np.load("./corpusEmbeddings.npy")

In [ ]:
corpusEmbeddings.shape

In [ ]:
# Agglomerative Clustering (Heirarchial Clustering)
clusteringModel = AgglomerativeClustering(n_clusters=None, distance_threshold=5.9)
clusteringModel.fit(corpusEmbeddings)
clusterAssignment = clusteringModel.labels_

In [ ]:
# processing clustered data
clusteredSentences = defaultdict(list)
clusteredUsers = defaultdict(set)
for sentenceId, clusterId in enumerate(clusterAssignment):
    clusteredSentences[clusterId].append(sentenceId)
    clusteredUsers[clusterId].add(botComments.username[sentenceId])

In [ ]:
len(clusteredUsers)

In [ ]:
botComments.message[clusteredSentences[2]]

In [ ]:
botComments.message[clusteredSentences[3]]

In [ ]:
botComments.message[clusteredSentences[100]]

In [ ]:
printer = [print(clusteredUsers[cluster], "\n") for cluster in clusteredUsers]

In [ ]:
# cluster based on same password
def clusterByPassword(botDF, userData):
  bots = botDF[['username']].drop_duplicates()
  botsWithDetails = pd.merge(botDF, userData, on='username')
  groupedBots = botsWithDetails.groupby(by='password_id').agg(set)
  expectedCluster = defaultdict(list)
  for id, cluster in enumerate(groupedBots.username.values):
    expectedCluster[id] = cluster
  return expectedCluster

In [ ]:
# defining function to get true/predicted edges
def getEdges(clusters):
  allEdges = set()
  for cluster in clusters.values():
    edges = set(combinations(cluster, 2))
    allEdges |= edges
  return allEdges

In [ ]:
# metric definition
def getMetric(n, trueEdges, predictedEdges):
  totalEdges = n*((n - 1)/2)
  trueEdges = set(map(tuple, map(sorted, trueEdges)))
  predictedEdges = set(map(tuple, map(sorted, predictedEdges)))
  truePositives = trueEdges & predictedEdges
  falsePositives = predictedEdges - trueEdges
  print("TE/PE:", len(trueEdges), len(predictedEdges))
  print("TP/FP:", len(truePositives), len(falsePositives))
  accuracy = len(truePositives)/len(trueEdges)
  penalizingFactor = 1 - ((len(falsePositives)*len(truePositives))/totalEdges)
  return accuracy*penalizingFactor

In [ ]:
expectedCluster = clusterByPassword(botComments, userData)

In [ ]:
trueEdges = getEdges(expectedCluster)

In [ ]:
predictedEdges = getEdges(clusteredUsers)

In [ ]:
getMetric(15163, trueEdges, predictedEdges)

In [ ]:
 pd.merge(botComments[['username']].drop_duplicates(), userData, on='username').shape

In [ ]:
corpusEmbeddings.shape

In [ ]:
botComments[['message']].drop_duplicates().shape

In [ ]:
botComments.shape

In [ ]:
np.unique(corpusEmbeddings, axis=0).shape